In [7]:
import pickle
import networkx as nx
import random

In [36]:
G=nx.read_edgelist('/Users/eneminova/LLM_Taxonomy/TaxonomyEnrichment/data/noun/all.edgelist', create_using=nx.DiGraph)

In [67]:
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorWithPadding, DefaultDataCollator
import torch

class GraphDataset(Dataset):
    def __init__(self, graph, num_triplets):
        self.graph = graph
        self.num_triplets = num_triplets
        self.triplets = self.generate_triplets()
        
    def generate_triplets(self):
        triplets = []
        while len(triplets) < self.num_triplets:
            child = random.choice(list(self.graph.nodes()))
            # print(child)
            parent = random.choice(list(self.graph.predecessors(child)))
            # print(parent)
            grandparent_candidates = list(self.graph.predecessors(parent))
            if child in grandparent_candidates:
                grandparent_candidates.remove(child)
            grandparent = random.choice(grandparent_candidates)
            triplet = (child, parent, grandparent)
            if triplets:
                if self.ensure_distance(triplets, triplet):
                    triplets.append(triplet)
                    print(triplets)
            else:
                triplets.append(triplet)
            # triplets.append(triplet)
        return triplets

    def ensure_distance(self, triplets, potential_triplet):
        print(triplets)
        print(potential_triplet)
        try: 
            first_len
        if len(nx.bidirectional_shortest_path(self.graph, triplets[-1][2], potential_triplet[0])) > 1:
            if len(nx.bidirectional_shortest_path(self.graph, potential_triplet[2], triplets[-1][0])) > 1:
                return True
            else:
                return False
        else:
            return False
        # for triplet in triplets:
        #     child, parent, grandparent = triplet
        #     if nx.has_path(self.graph, child, parent) and nx.has_path(self.graph, parent, grandparent):
        #         continue
        #     else:
        #         return False
        # return True

    def __len__(self):
        return self.num_triplets

    def __getitem__(self, idx):
        child, parent, grandparent = self.triplets[idx]
        return (child, parent, grandparent)

class GraphDataCollator(DefaultDataCollator):
    def __call__(self, features):

        final_pairs = []
        # Разделяем каждый триплет на два дуплета

        for feature in features:
            (child, parent, grandparent) = feature
            final_pairs.append(((child, parent), (parent, grandparent)))

        return final_pairs


# Задаем параметры
num_triplets = 128
batch_size = 32

# Создаем датасет
dataset = GraphDataset(G, num_triplets)

# Создаем DataLoader и DataCollator
data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=GraphDataCollator())

# Пример использования DataLoader
for batch in data_loader:
    # print("Child-Parent Batch:")
    print(*batch, sep='\n')
    print(len(batch))
    # print("Parent-Grandparent Batch:")
    # print(batch_parent_grandparent)


[('lot.n.02', 'tract.n.01', 'geographical_area.n.01')]
('city.n.03', 'municipality.n.02', 'gathering.n.01')


NetworkXNoPath: No path between geographical_area.n.01 and city.n.03.

In [42]:
list(G.edges())[-1]

('kaaba.n.01', 'test.test.536')